# Pytorch Basics

In [1]:
# import packages
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

## 1. Basic autograd example 1 

In [2]:
# Create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [3]:
# Build a computational graph.
y = w * x + b

In [4]:
# Compute gradients.
y.backward()

In [5]:
# Print out the gradients.
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


## 2. Basic autograd example 2

In [6]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [7]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.3604, -0.2018,  0.0575],
        [-0.3579,  0.0005, -0.5019]], requires_grad=True)
b:  Parameter containing:
tensor([-0.2982,  0.4262], requires_grad=True)


In [8]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [9]:
# Forward pass.
pred = linear(x)

In [10]:
# Compute loss
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  1.5350843667984009


In [11]:
# Backward pass.
loss.backward()

In [12]:
# Print out the gradients.
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[-0.9874,  0.0574,  0.6015],
        [-0.2146,  0.0407, -0.3846]])
dL/db:  tensor([-0.0740,  0.5068])


In [13]:
# 1-step gradient descent.
optimizer.step()

In [14]:
# Perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [15]:
# Print out the loss after 1-step gradient descent.
pred = linear(x) # Optimizer make it change.
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.517189383506775


## 3. Loading data from numpy

In [16]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

In [17]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

In [18]:
# Convert the torch tensor to a numpy array.
z = y.numpy()

## 4. Input pipeline

In [19]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='data/CIFAR/', train=True, transform=transforms.ToTensor(), download=True) # ‘../’为向上跳一级

100.0%

Extracting data/CIFAR/cifar-10-python.tar.gz to data/CIFAR/


In [20]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [21]:
# Data loader (this provides quenes and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [22]:
# When iteration starts, quene and thread start to load data from files.
data_iter = iter(train_loader)

In [23]:
# Mini-batch images and labels.
images, labels = data_iter.next()

In [24]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here
    pass

## 5. Input pipeline for custom dataset

In [25]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # Initialize file paths or a list of file names.
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 1

In [26]:
# You can then use the prebuilt data loader.
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset, batch_size=64, shuffle=True)

## 6. Pretrained model

In [27]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

In [28]:
# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

In [29]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

In [30]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size()) 

KeyboardInterrupt: 

## 7. Save and load the model

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [ ]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))